## Laboratorio 8    
Gabriel García - 21352     
Diego Leiva   
Pablo Orellana

## Task 1
1. Investigar el algoritmo AC-3 y su relacion con el algoritmo de backtracking search       

El algormitmo AC-3 (Algoritmo de conssitencia de Arcos-3) Se utiliza en resolucion de problemas de restricción para reducir el dominio de las variables. Se centra en eliminar los valores incosistentes de las variables basandose en restricciones binarias. AC-3 se relaciona con el algoritmo de backtracking search en el sentido de que AC-3 se utiliza a menudo con el preprocesamiento para reducir el espacio de busqueda antes de aplicar backtracking search. AC-3 ayuda a identificar y eliminar las inconsistencias temprano, lo que puede reducir significativamente el tiempo de busqueda en el algoritmo de backtracking (GeeksforGeeks, 2023).     

2. Defina en sus propias palabras el termino "Arc Consistency"     

Arc Consistency se refiere a la propiedad en un problema de restricción donde los valores en el dominio de una variable son consistentes con las restricciones binarias que tienen con otras variables. Es decir, un arco (una conexion entre dos variables) se considera consistente si para el valor en el dominio de la variable A, hay al menos un valor en el dominio de la variable B que satisface la restricción binaria entre A y B. Por lo que podemos concluir que "Arc Consistency" es una medida de que tan restringido es el dominio de una variable basándose en las restricciones binarias que tienen con otras variables en un problema de restricción (British Columbia University, 2009).  



Referencias
- GeeksforGeeks. (2023, 20 de octubre). Backtracking Algorithms. GeeksforGeeks. https://www.geeksforgeeks.org/backtracking-algorithms/

- British Columbia University. (2009, 10). CSP3: Backtracking Search. Department of Computer Science. https://www.cs.ubc.ca/~kevinlb/teaching/cs322%20-%202009-10/Lectures/CSP3.pdf

## Task 2